In [1]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [2]:
data_dir = 'data'

In [3]:
import pathlib
data_dir = pathlib.Path(data_dir)
data_dir

WindowsPath('data')

In [4]:
len(list(data_dir.glob('*/*.png')))

325

In [5]:
pose_name_dict = {
    'Adho Mukha Svanasana':list(data_dir.glob('Adho Mukha Svanasana/*')),
    'Adho Mukha Vrksasana':list(data_dir.glob('Adho Mukha Vrksasana/*')),
    'Alanasana':list(data_dir.glob('Alanasana/*')),
    'Anjaneyasana':list(data_dir.glob('Anjaneyasana/*')),
    'Ardha Chandrasana':list(data_dir.glob('Ardha Chandrasana/*')),
}

In [6]:
pose_label_dict = {
    'Adho Mukha Svanasana':0,
    'Adho Mukha Vrksasana':1,
    'Alanasana':2,
    'Anjaneyasana':3,
    'Ardha Chandrasana':4,
}

In [7]:
X, y = [], []

for yoga_name, images in pose_name_dict.items():
    for image in images:
        img = cv2.imread(str(image))
        resized_img = cv2.resize(img, (128, 128))
        X.append(resized_img)
        y.append(pose_label_dict[yoga_name])

X = np.array(X)
y = np.array(y)

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0)

In [9]:
X_train = X_train / 255
X_test = X_test / 255
print(len(X_test), len(X_train))

29 258


In [10]:
data_augmentation = keras.Sequential(
  [
    layers.experimental.preprocessing.RandomFlip("horizontal", 
                                                 input_shape=(128, 
                                                              128,
                                                              3)),
    layers.experimental.preprocessing.RandomRotation(0.2),
    layers.experimental.preprocessing.RandomZoom(0.2),
  ]
)

In [11]:
num_classes = 5

cnn = Sequential([
    data_augmentation,
    layers.Conv2D(filters=16, kernel_size=3, activation='relu', padding='same'),
    layers.MaxPool2D(pool_size=2, strides=2),
    layers.Conv2D(filters=32, kernel_size=3, activation='relu', padding='same'),
    layers.MaxPool2D(pool_size=2, strides=2),
    layers.Conv2D(filters=64, kernel_size=3, activation='relu', padding='same'),
    layers.MaxPool2D(pool_size=2, strides=2),
    layers.Flatten(),
    
    layers.Dense(256, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])

In [12]:
cnn.compile(optimizer='Adam',
            loss=tf.keras.losses.SparseCategoricalCrossentropy(),
            metrics=["accuracy"])

In [13]:
cnn.fit(X_train, y_train, epochs=60)

Epoch 1/60
9/9 [==============================] - 4s 154ms/step - loss: 1.7095 - accuracy: 0.2558
Epoch 2/60
9/9 [==============================] - 1s 148ms/step - loss: 1.5462 - accuracy: 0.2636
Epoch 3/60
9/9 [==============================] - 1s 150ms/step - loss: 1.5352 - accuracy: 0.2829
Epoch 4/60
9/9 [==============================] - 1s 149ms/step - loss: 1.5169 - accuracy: 0.2791
Epoch 5/60
9/9 [==============================] - 1s 150ms/step - loss: 1.4404 - accuracy: 0.3837
Epoch 6/60
9/9 [==============================] - 1s 150ms/step - loss: 1.4186 - accuracy: 0.3915
Epoch 7/60
9/9 [==============================] - 1s 151ms/step - loss: 1.3909 - accuracy: 0.3527
Epoch 8/60
9/9 [==============================] - 1s 149ms/step - loss: 1.2796 - accuracy: 0.4535
Epoch 9/60
9/9 [==============================] - 1s 150ms/step - loss: 1.3467 - accuracy: 0.4496
Epoch 10/60
9/9 [==============================] - 1s 147ms/step - loss: 1.3219 - accuracy: 0.4264
Epoch 11/60
9/9 [==

In [14]:
cnn.evaluate(X_test, y_test)

1/1 [==============================] - 0s 354ms/step - loss: 0.7370 - accuracy: 0.7586


[0.7370356917381287, 0.7586206793785095]

In [15]:
import numpy as np
from keras.preprocessing import image
test_image = image.load_img('svana.jpg', target_size = (128, 128))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image)
result

1/1 [==============================] - 0s 185ms/step


array([[2.0152922e-29, 0.0000000e+00, 1.0000000e+00, 0.0000000e+00,
        0.0000000e+00]], dtype=float32)

In [16]:
list_of_poses = ["Adho Mukha Svanasana", "Adho Mukha Vrksasana", "Alanasana", 
                 "Anjaneyasana", "Ardha Chandrasana"]
# import tensorflow as tf

# logits = tf.constant(result)
# print(logits)

# # Apply softmax
# probabilities = tf.nn.softmax(logits)

# print(probabilities.numpy())
# idx = np.argmax(probabilities[0])
# print(list_of_poses[idx])

idx = np.argmax(result[0])
print(list_of_poses[idx])


Alanasana
